In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import (
    read_sql_query, 
    DataFrame,
    ExcelWriter
)
from enum import Enum
import openpyxl




In [2]:
class Period(Enum):
    date = '2021-10-01'
    wb=''
    date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [3]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query_presenceQty = f'''
SELECT 
    a.*, COUNT(*) AS qte_pres_by_creator
FROM
    (SELECT 
            dgs.created_by,
            au.email AS email,
            au.username
    FROM
        caris_db.dream_group_session dgs
    LEFT JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
    LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
GROUP BY a.created_by
'''

query_beneficiariesQty = f'''
SELECT *, COUNT(*)AS qte_by_creator FROM
    (SELECT
        dgs.id AS id_session,
            topic,
            date,
            dga.id_patient AS id_patient,
            dgs.created_by,
            lc.name AS commune,
            ld.name AS departement,
            dh.name AS hub,
            au.email AS email,
            au.username,
            b.first_name,
            b.last_name,
            p.patient_code as dreams_code
    FROM
        caris_db.dream_group_session dgs
    INNER JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
        LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    left join beneficiary b on dga.id_patient=b.id_patient
    left join patient p on p.id=dga.id_patient
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
            group by a.id_patient, a.username
'''

query_parentingBene = f'''
SELECT 
    *, COUNT(*) AS qte_by_creator
FROM
    (SELECT 
        dpgs.id_group AS id_session,
            topic,
            date,
            dpga.id_patient AS id_patient,
            dpgs.created_by,
            lc.name AS commune,
            ld.name AS departement,
            dh.name AS hub,
            au.email AS email,
            au.username,
            b.first_name,
            b.last_name,
            p.patient_code AS dreams_code
    FROM
        caris_db.dream_parenting_group_session dpgs
    LEFT JOIN dream_parenting_group_attendance dpga ON dpgs.id = dpga.id_parenting_group_session
    LEFT JOIN dream_group dg ON dpgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dpgs.created_by
    LEFT JOIN beneficiary b ON dpga.id_patient = b.id_patient
    LEFT JOIN patient p ON p.id = dpga.id_patient
    WHERE
        ((dpga.parent_g = 'P')
            OR (dpga.parent_vd = 'P')
            OR (dpga.yg_g = 'P')
            OR (dpga.yg_vd = 'P'))
            AND (dpgs.date >= '{Period.date.value}')) a
GROUP BY a.id_patient , a.username
'''

query_parentingQty = f'''
SELECT 
    a.*, COUNT(*) AS qte_by_creator
FROM
    (SELECT 
        dpgs.created_by, au.email AS email, au.username
    FROM
        caris_db.dream_parenting_group_session dpgs
    LEFT JOIN dream_parenting_group_attendance dpga ON dpgs.id = dpga.id_parenting_group_session
    LEFT JOIN dream_group dg ON dpgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dpgs.created_by
    WHERE ((dpga.parent_g = 'P') OR (dpga.parent_vd= 'P') OR (dpga.yg_g='P') OR (dpga.yg_vd='P') )
            AND (dpgs.date >= '{Period.date.value}') ) a
GROUP BY a.created_by
'''




pres_byAgent= read_sql_query(query_presenceQty, engine, parse_dates=True)
ben_byAgent = read_sql_query(query_beneficiariesQty, engine, parse_dates=True)

pres_parenting= read_sql_query(query_parentingQty, engine, parse_dates=True)
parenting_byAgent = read_sql_query(query_parentingBene, engine, parse_dates=True)


# close the pool of connection
engine.dispose()


True

In [4]:
pres_byAgent.rename(columns={'qte_pres_by_creator':'nombre_pres'},inplace=True)
ben_byAgent.rename(columns={'qte_by_creator':'nombre_pres'},inplace=True)
pres_parenting.rename(columns={'qte_by_creator':'nombre_pres'},inplace=True)
parenting_byAgent.rename(columns={'qte_by_creator':'nombre_pres'},inplace=True)

In [5]:
print('Nombre de presence totale: ',pres_byAgent.nombre_pres.sum())
print('Nombre de presence totale: ',ben_byAgent.nombre_pres.sum())
print('Nombre de filles: ',ben_byAgent.id_patient.count())
print('Nombre de presence totale parenting: ',pres_parenting.nombre_pres.sum())
print('Nombre de presence totale parenting: ',parenting_byAgent.nombre_pres.sum())
print('Nombre de filles: ',parenting_byAgent.id_patient.count())

Nombre de presence totale:  42552
Nombre de presence totale:  42552
Nombre de filles:  2615
Nombre de presence totale parenting:  244
Nombre de presence totale parenting:  244
Nombre de filles:  86


# First Component

In [6]:
pres_byAgent = pres_byAgent[['username','email','nombre_pres']]
pres_byAgent

print('---------------------------------------------------------------------------------------------')

pres_parenting = pres_parenting[['username','email','nombre_pres']]
pres_parenting

,username,email,nombre_pres
0,johanejules,johane.jules@carisfoundationintl.org,16
1,fbyron,byronfabrice9@gmail.com,4258
2,billyjolicoeur,billyjolicoeur95@gmail.com,7740
3,FredlyEliona,elionafredly7@gmail.com,3453
4,dchelot,blonnichel@yahoo.fr,1774
5,pggd1234,gerald.poteau@carisfoundationintl.org,2485
6,Armando,diegoarmando.dorlean@carisfoundationintl.org,9392
7,Ernstbelabre,ernst.belabre@carisfoundationintl.org,3054
8,France14,francenelstjean.saed@yahoo.com,2885
9,despeignes,roselord.despeignes@carisfoundationintl.org,879


---------------------------------------------------------------------------------------------


,username,email,nombre_pres
0,fbyron,byronfabrice9@gmail.com,217
1,France14,francenelstjean.saed@yahoo.com,27


In [7]:
def validation_pivot(pt):
    if not pt.empty:
        pt = pt.pivot_table(values='id_patient',index='username',columns='commune',aggfunc='count',fill_value=0,margins=True, margins_name='Total')
    if pt.empty:
        pt =  DataFrame({"resultats":["No data entry"]})
    return pt

# Component 2

In [8]:
#ben_table = ben_byAgent.pivot_table(values='id_patient',index='username',columns='commune',aggfunc='count',fill_value=0,margins=True, margins_name='Total')
ben_table = validation_pivot(ben_byAgent)
ben_table

print('-----------------------------------------------------------------------------------------------------')

#parenting_table = parenting_byAgent.pivot_table(values='id_patient',index='username',columns='commune',aggfunc='count',fill_value=0,margins=True, margins_name='Total')
parenting_table = validation_pivot(parenting_byAgent)
parenting_table


commune,Cap-Haïtien,Carrefour,Delmas,Desdunes,Dessalines,Grande Saline,Kenscoff,La Chapelle,Liancourt,Limonade,Montrouis,Petite Rivière de l'Artibonite,Plaine-du-Nord,Port-au-Prince,Pétionville,Quartier-Morin,Saint-Marc,Tabarre,Verrettes,Total
username,,,,,,,,,,,,,,,,,,,,
Armando,0,0,0,0,0,0,0,0,29,0,0,0,0,0,0,0,8,0,323,360
BonWoodRidchard,0,65,49,0,0,51,0,0,0,0,50,0,0,239,127,0,0,16,0,597
Ernstbelabre,0,0,0,0,19,30,0,0,0,0,0,0,0,0,0,0,118,0,26,193
France14,0,0,0,18,177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,195
FredlyEliona,0,0,153,8,0,0,30,0,0,0,0,22,0,56,11,0,0,0,0,280
billyjolicoeur,33,0,0,0,0,0,273,0,0,20,0,0,2,16,52,0,0,7,0,403
dchelot,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,97,0,12,134
despeignes,0,0,0,0,0,0,0,19,0,0,0,0,0,0,0,0,3,0,40,62
fbyron,26,6,0,5,0,0,26,0,0,0,0,83,0,94,24,0,0,9,0,273


-----------------------------------------------------------------------------------------------------


commune,Cap-Haïtien,Dessalines,Gressier,Liancourt,Limonade,Petite Rivière de l'Artibonite,Total
username,,,,,,,
France14,0,27,0,0,0,0,27
fbyron,26,0,11,10,5,7,59
Total,26,27,11,10,5,7,86


# 3 component 

In [9]:
curriculum_result = DataFrame(
    [
        {'Nombre de fille': ben_byAgent.id_patient.count(),
        'Nombre de presence': pres_byAgent.nombre_pres.sum()}
    ]
)
curriculum_result

print('-----------------------------------------------------------------------')

parenting_result = DataFrame(
    [
        {'Nombre de fille/parent parenting': parenting_byAgent.id_patient.count(),
        'Nombre de presence': pres_parenting.nombre_pres.sum()}
    ]
)
parenting_result


,Nombre de fille,Nombre de presence
0,2615,42552


-----------------------------------------------------------------------


,Nombre de fille/parent parenting,Nombre de presence
0,86,244


# Report

In [10]:
all_report = ExcelWriter(f"rapport_performance.xlsx",engine="openpyxl")
curriculum_result.to_excel(all_report,sheet_name="resultat_curriculum",index=False,na_rep="")
pres_byAgent.to_excel(all_report,sheet_name="presence_agent_curriculum",index=False,na_rep="")
ben_table.to_excel(all_report,sheet_name="ben_communes_curriculum",index=True,na_rep="")
parenting_result.to_excel(all_report,sheet_name="resultat_parenting",index=False,na_rep="")
pres_parenting.to_excel(all_report,sheet_name="pres_agent_parenting",index=False,na_rep="")
parenting_table.to_excel(all_report,sheet_name="ben_communes_parenting",index=True,na_rep="")
all_report.save()